In [1]:
# label for gender
FEMALE = 0
MALE = 1
UNKNOWN = -1

def mapGenderToTag(gender):
    if gender == 'female':
        return FEMALE
    elif gender == 'male':
        return MALE
    else:
        return UNKNOWN

In [2]:
# stripe HTML from 知乎 answers
from bs4 import BeautifulSoup
def concatAndStriptHTMLFromContent(aaa):
    content, headline, gender = aaa
    soup = BeautifulSoup(content)
    return [headline + ' ' + soup.get_text(), mapGenderToTag(gender)]

In [8]:
# fetch data from crawled answers
import sqlite3

# prepare data
conn = sqlite3.connect('/Users/linonetwo/Desktop/repo/zhihu-nlp-playground/data/questionsAnswers.sqlite')

def answerList(gender, count, order):
    cursor = conn.execute(f"""
        SELECT answers.content, users.headline, users.gender FROM answers LEFT JOIN users
        On answers.user = users.id
        WHERE users.gender = "{gender}"
        ORDER BY answers._id {order}
        LIMIT {count}
    """)
    return list(map(concatAndStriptHTMLFromContent, cursor.fetchall()))

In [9]:
# prepare data for training
import random
import pandas as pd

size = 2
train_data = answerList('male', size, 'DESC') + answerList('female', size, 'DESC')
eval_data = answerList('male', size, 'ASC') + answerList('female', size, 'ASC')

random.shuffle(train_data)
random.shuffle(eval_data)

# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. The first column is the text with type str, and the second column is the label with type int.
train_df = pd.DataFrame(train_data, columns=['text', 'labels'])
eval_df = pd.DataFrame(eval_data)

In [10]:
train_df.head()

,text,labels
0,温柔而锋利 尊重女性将心比心热爱生活保持自醒,0
1,地理科学爱好者 有责任心比爱我更重要,0
2,键盘侠/KPOP爱好者/不入流的小辩手 如果只说主观的话那没有什么用，标准都是自己定的什么叫...,1
3,小肥虾 小怪兽 BANANA 我 男生喜欢那种160以上175以下的女生（觉得很像金香起吧...,1


In [11]:
eval_df.head()

,0,1
0,等风等雨在等你 俗话说“千里姻缘一线牵”，写这个帖子就是给未来老婆多一条找到我的途径；最好的...,1
1,念经念经 刚考完研，本科是一本，考研是好一点的一本，没啥大的追求，嘻嘻，学校在上海，大概率能...,0
2,琴心剑魄今何在 91年魔都土著，普通一本工科毕业，JAVA程序猿一只，平时喜欢健身运动，摄影...,1
3,不知我者谓我何求 2020年打卡工具人签个到⁽˙⁸˙⁾ʹ˜˜Ummm 2019年比较流年不利...,0


In [ ]:
from simpletransformers.classification import ClassificationModel
import sklearn


# Create a ClassificationModel
train_args={
    'sliding_window': True,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'evaluate_during_training': True,
    'no_cache': False,
    'logging_steps': 5,
    'stride': 0.8,
    'max_seq_length': 512
}
model = ClassificationModel('bert', 'bert-base-chinese', use_cuda=False, args=train_args) # You can set class weights by using the optional weight argument

# Train the model
model.train_model(train_df, eval_df=eval_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

Converting to features started. Cache is not used.
sliding_window enabled



4 features created from 4 samples.


Running loss: 0.865374
Converting to features started. Cache is not used.
sliding_window enabled


/opt/anaconda3/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:686: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



4 features created from 4 samples.
